In [29]:
!git clean -df

Removing .ipynb_checkpoints/
Removing B1F4/
Removing poly-direct.cpp
Removing poly-grd.maple
Removing poly-model.h
Removing poly-nogrd.maple
Removing polynomial_generation/src/.poly-gen_mb-nrg.pl.swp
Removing run.ipynb
Removing vars.cpp


Create a folder with input file

In [30]:
%mkdir B1F4

In [31]:
%cd B1F4

/home/azonca/Paesani/potential_fitting/B1F4


In [33]:
%cp /home/azonca/Paesani/potential_fitting/polynomial_generation/tests/1B/bf4.in .

In [37]:
%%file config.ini

[polynomial_generation]
molecule_definition = bf4.in
polynomial_order = 4

Overwriting config.ini


In [38]:
!ls

bf4.in	config.ini


In [39]:
# have to run from the folder
config_filename = "config.ini"

In [40]:
import configparser
import os

config = configparser.ConfigParser()
config.read(config_filename)

['config.ini']

# Polynomial generation

### Define input file

### Create maple input files

In [43]:
order = config.getint("polynomial_generation", "polynomial_order")

In [44]:
input_file = os.path.join(root_folder, config.get("polynomial_generation", "molecule_definition"))

In [45]:
os.mkdir("polynomial_generation")

In [46]:
os.chdir("polynomial_generation")

In [49]:
potential_fitting_root_folder = "/home/azonca/Paesani/potential_fitting/"

In [50]:
%pwd

'/home/azonca/Paesani/potential_fitting/B1F4/polynomial_generation'

In [53]:
!$potential_fitting_root_folder/polynomial_generation/src/poly-gen_mb-nrg.pl $order ../$input_file > poly_generation.log 

In [54]:
ls

poly-direct.cpp      poly-grd.maple  poly-nogrd.maple
poly_generation.log  poly-model.h    vars.cpp


### Run maple and transform output to C++

In [55]:
for f in ["poly-nogrd", "poly-grd"]:
    maple_file = f + ".maple"
    !maple $maple_file
    c_file = f + ".c"
    cpp_file = f + ".cpp"
    !$potential_fitting_root_folder/polynomial_generation/src/clean-maple-c.pl < $c_file > $cpp_file

    |\^/|     Maple 2016 (X86 64 LINUX)
._|\|   |/|_. Copyright (c) Maplesoft, a division of Waterloo Maple Inc. 2016
 \  MAPLE  /  All rights reserved. Maple is a trademark of
 <____ ____>  Waterloo Maple Inc.
      |       Type ? for help.
>     p[1] := x10+x08+x06+x07+x09+x05:
>     p[2] := x04+x02+x01+x03:
> 
>     p[3] := x09*x10+x08*x10+x05*x06+x06*x07+x08*x09+x05*x07+x05*x08+x06*x10+x07*x10+x06*x08+x07*x09+x05*x09:
>     p[4] := x03*x07+x01*x10+x01*x09+x04*x05+x02*x06+x02*x10+x01*x08+x04*x08+x03*x05+x04*x06+x02*x07+x03*x09:
>     p[5] := x01*x05+x02*x05+x04*x10+x04*x09+x03*x06+x02*x09+x02*x08+x03*x08+x01*x07+x01*x06+x03*x10+x04*x07:
>     p[6] := x07*x08+x06*x09+x05*x10:
>     p[7] := x04*x04+x01*x01+x02*x02+x03*x03:
>     p[8] := x08*x08+x07*x07+x05*x05+x10*x10+x09*x09+x06*x06:
>     p[9] := x01*x02+x03*x04+x01*x03+x02*x03+x01*x04+x02*x04:
> 
>     p[10] := x04*x06*x08+x02*x06*x10+x01*x09*x10+x04*x05*x06+x03*x05*x07+x02*x07*x10+x04*x05*x08+x01*x08*x09+x03*x07*x09+x02*x06*x07+x0

# Fitting

In [56]:
os.chdir("..")

In [57]:
os.mkdir("fitting")

In [58]:
os.chdir("fitting")

### Setup fitting and compile

In [59]:
!$potential_fitting_root_folder/fitting/1B/get_codes/setup.ba ../polynomial_generation/poly-grd.cpp ../polynomial_generation/poly-nogrd.cpp ../polynomial_generation/poly-direct.cpp ../polynomial_generation/poly-model.h ../$input_file $order 0 .

/home/azonca/Paesani/potential_fitting/B1F4/fitting/../templates/normal
B1F4
cp: cannot stat ‘/home/azonca/Paesani/potential_fitting/B1F4/fitting/../templates/normal/*’: No such file or directory
sed: can't read eval-1b-pot.cpp: No such file or directory
sed: can't read eval-1b-pot-wgrads.cpp: No such file or directory
sed: can't read fit-1b-polynomials.cpp: No such file or directory
sed: can't read stuff.h: No such file or directory
sed: can't read stuff.cpp: No such file or directory
sed: can't read training-set.cpp: No such file or directory
sed: can't read training-set.h: No such file or directory
sed: can't read fit-utils.cpp: No such file or directory
sed: can't read fit-utils.h: No such file or directory
sed: can't read x1b-v1.cpp: No such file or directory
sed: can't read x1b-v1.h: No such file or directory
sed: can't read x1b-v1x.cpp: No such file or directory
sed: can't read x1b-v1x.h: No such file or directory
sed: can't read eval-1b-pot-wgrads.cpp: No such file or directory

### Run fitting

First make sure that the `.xyz` training set is available

In [ ]:
%cd B1F4/normal/4

In [ ]:
!ln -s  ~/Paesani/bf4_training_set.xyz .

In [ ]:
!./fit-1b-polynomials *_training_set.xyz > fit-1b-polynomials.log

Check fitted output coefficients

In [ ]:
!cat fit-x1b-v1.dat